## Model Fitting and Parameter Tuning

In summary, XGBoost is the absolute winner; Gradient Boost has great performance, but takes too long to run.

4-fold Corss-Validation.
Metric: AUC (Area under the ROC curve)

In [4]:
import os
import pandas as pd
import numpy as np

In [5]:
os.chdir('C:/Users/fando/Box Sync/Springboard/Capstone Project/Data')

In [6]:
dtype_dict_ready = np.load('dtype_dict_ready.npy').item()
allData = pd.read_csv('allData_ready.txt', sep=" ", dtype = dtype_dict_ready, encoding = "ISO-8859-1")

In [7]:
# book_day is not very informative at current stage
allData.drop(['book_day'], axis=1, inplace=True)
# allData = allData.drop_duplicates(

In [5]:
allData.shape

(286558, 124)

In [8]:
X = allData.drop('target', axis=1)
# Reclass y (availability rate in 60 days) into 5 categories
y = allData['target']
#y = pd.cut(y, 4, labels=['1','2','3','4'])

## Lasso
### Cross-validation

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
import operator

In [10]:
cross_val_score(LogisticRegression(penalty='l1'), X, y, scoring='roc_auc', cv=5, n_jobs=-1)

array([ 0.71546945,  0.69149803,  0.7436569 ,  0.65860104,  0.71314673])

In [14]:
np.array([ 0.71546945,  0.69149803,  0.7436569 ,  0.65860104,  0.71314673]).mean()

0.70447442999999998

### Test-train split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [18]:
clf = LogisticRegression(penalty='l1')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
clf.score(X_test, y_test)

0.74001954215522059

In [20]:
y_pred = clf.predict(X_test)

In [48]:
y_pred = list(map(int, y_pred))

In [49]:
y_int = list(map(int, y_test))

In [50]:
sum(list(map(operator.sub, y_int, y_pred)))/len(y_int)

0.12828059847927398

## Stochastic Gradient Descent

In [10]:
from sklearn.linear_model import SGDClassifier

In [17]:
cross_val_score(SGDClassifier(penalty='l1'), X, y, scoring='roc_auc', cv=5)

array([ 0.6358725 ,  0.63436149,  0.63980129,  0.54611575,  0.57431357])

## Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
cross_val_score(RandomForestClassifier(max_depth=20, n_estimators = 75, n_jobs=-1), X, y, scoring='roc_auc', cv=4).mean()

0.78248512408530202

In [9]:
for k in list(range(30,80,5)):
    for d in list(range(10, 50, 5)):
        auc = cross_val_score(RandomForestClassifier(max_depth=d, n_estimators = k, n_jobs=-1), X, y, scoring='roc_auc', cv=4).mean()
        print(k, d, auc)
# Choose k=75, d=20

30 10 0.763275534632
30 15 0.782791877748
30 20 0.786430680922
30 25 0.781405087699
30 30 0.777916950733
30 35 0.776023986414
30 40 0.772516918143
30 45 0.773973810838
35 10 0.777210378232
35 15 0.783317879384
35 20 0.782959919378
35 25 0.780824591007
35 30 0.773497086865
35 35 0.771065598894
35 40 0.772072390697
35 45 0.776852113063
40 10 0.775926103045
40 15 0.791829856963
40 20 0.778002977831
40 25 0.77810481773
40 30 0.779550450258
40 35 0.776256428578
40 40 0.775786840224
40 45 0.774935283625
45 10 0.777327462626
45 15 0.787682979614
45 20 0.777340638965
45 25 0.778564095249
45 30 0.777526991538
45 35 0.779606055962
45 40 0.775516977888
45 45 0.777252596271
50 10 0.770568704373
50 15 0.786928539981
50 20 0.788957469616
50 25 0.778455508532
50 30 0.78177628917
50 35 0.775065677057
50 40 0.773184379214
50 45 0.780792076615
55 10 0.770085711137
55 15 0.783798682933
55 20 0.792280587208
55 25 0.773078568215
55 30 0.775889540643
55 35 0.775856154855
55 40 0.779346816862
55 45 0.7733205

## Neural Network 

In [19]:
from sklearn.neural_network import MLPClassifier

In [20]:
for k in list(range(80, 300, 20)):
    print('Hidden layer', k, 
          cross_val_score(MLPClassifier(activation='logistic', solver = 'adam', hidden_layer_sizes=k), X, y, scoring='roc_auc', cv=4).mean())

Hidden layer 80 0.735707632707
Hidden layer 100 0.728133481026
Hidden layer 120 0.767088611988
Hidden layer 140 0.757766541289
Hidden layer 160 0.752106550008
Hidden layer 180 0.756451038877
Hidden layer 200 0.760403292035
Hidden layer 220 0.733977041521
Hidden layer 240 0.766025457636
Hidden layer 260 0.78022291826
Hidden layer 280 0.747326213614


## Gradient Boosting
### Great performance. But take too long to run.

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
cross_val_score(GradientBoostingClassifier(n_estimators = 80, max_depth = 20), X, y, scoring='roc_auc', cv=5).mean()

0.81021767736920258

In [14]:
#for k in list(range(80,160,20)):
#    for d in list(range(10, 30, 5)):
#        auc = cross_val_score(GradientBoostingClassifier(max_depth=d, n_estimators = k), X, y, scoring='roc_auc', cv=4).mean()
#        print(k, d, auc)

Each model takes 4~5 hours to fit. Part of the result as follows:

n_estimator /      max_depth   /  auc

80 / 10 / 0.916182837347

80 / 15 / 0.904932929805

80 / 20 / 0.884332661843

80 / 25 / 0.842941079002

100 / 10 / 0.931677630294

100 / 15 / 0.920015544293

100 / 20 / 0.896099175263

## Extra Tree

In [30]:
from sklearn.ensemble import ExtraTreesClassifier

In [31]:
for k in list(range(60,100,10)):
    for d in list(range(10, 30, 5)):
        auc = cross_val_score(ExtraTreesClassifier(max_depth=d, n_estimators = k, n_jobs=-1), X, y, scoring='roc_auc', cv=4).mean()
        print(k, d, auc)

60 10 0.767212973866
60 15 0.770796047573
60 20 0.759181343776
60 25 0.751967265095
70 10 0.770806067879
70 15 0.776094021291
70 20 0.758918227616
70 25 0.749162845849
80 10 0.777557665921
80 15 0.769113365841
80 20 0.764171339524
80 25 0.753994470822
90 10 0.773093230655
90 15 0.774660727016
90 20 0.759655421914
90 25 0.754216057858


## Support Vector Classifier

In [7]:
from sklearn.svm import SVC

In [ ]:
for k in ['rbf', 'linear', 'sigmoid']:
    print(k, cross_val_score(SVC(kernel = k), X, y, scoring='roc_auc', cv=4).mean())

## AdaBoost

In [8]:
from sklearn.ensemble import AdaBoostClassifier

In [12]:
for k in list(range(80, 300, 20)):
    print('n_estimator', k, 
        cross_val_score(AdaBoostClassifier(n_estimators = k), X, y, scoring='roc_auc', cv=4).mean())

n_estimator 80 0.710933042073
n_estimator 100 0.718267452239
n_estimator 120 0.7265857633
n_estimator 140 0.728670378638
n_estimator 160 0.73303407939
n_estimator 180 0.737225616036
n_estimator 200 0.740981516158
n_estimator 220 0.744554854985
n_estimator 240 0.746123075021
n_estimator 260 0.748678187266
n_estimator 280 0.752256992835


In [13]:
for k in list(range(300, 500, 20)):
    print('n_estimator', k, 
        cross_val_score(AdaBoostClassifier(n_estimators = k), X, y, scoring='roc_auc', cv=4).mean())

n_estimator 300 0.755230309208
n_estimator 320 0.755315128827
n_estimator 340 0.757435136425
n_estimator 360 0.760370489708
n_estimator 380 0.762245249205
n_estimator 400 0.765559622379
n_estimator 420 0.766644567065
n_estimator 440 0.768595058263
n_estimator 460 0.771087887791
n_estimator 480 0.773205399469


## XGBoost
### The Winner.

In [1]:
import xgboost as xgb

In [2]:
def to_int(df, var):
    df[var] = df[var].astype(int)
    return df

In [9]:
for var in ['book_month', 'book_year', 'book_weekday']:
    X = to_int(X, var)

In [13]:
for k in list(range(80, 180, 20)):
    for d in list(range(10, 30, 5)):
        print('n_estimator', k, 'max_depth', d, 
            cross_val_score(xgb.XGBClassifier(n_estimators= k, max_depth= d, objective= 'binary:logistic', n_jobs= -1), X, y, scoring='roc_auc', cv=4).mean())

n_estimator 80 max_depth 10 0.916437200856
n_estimator 80 max_depth 15 0.919435447259
n_estimator 80 max_depth 20 0.902666818388
n_estimator 80 max_depth 25 0.885486827609
n_estimator 100 max_depth 10 0.932993446032
n_estimator 100 max_depth 15 0.929901669888
n_estimator 100 max_depth 20 0.913411131735
n_estimator 100 max_depth 25 0.90185555106
n_estimator 120 max_depth 10 0.942786633958
n_estimator 120 max_depth 15 0.936951855256
n_estimator 120 max_depth 20 0.921449752209
n_estimator 120 max_depth 25 0.910276201444
n_estimator 140 max_depth 10 0.951236838343
n_estimator 140 max_depth 15 0.94252209576
n_estimator 140 max_depth 20 0.92632364823
n_estimator 140 max_depth 25 0.918085916629
n_estimator 160 max_depth 10 0.958515959299
n_estimator 160 max_depth 15 0.949059410791
n_estimator 160 max_depth 20 0.9319685603
n_estimator 160 max_depth 25 0.923289436204
